In [1]:
# надо вывести топ 3 команд которые отправили больше всех СМСок за 2019 год

In [2]:
import pandas as pd 
import numpy as np
import pyarrow.parquet as pq
from pandas import read_csv, DataFrame
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [3]:
# df = pq.read_parquet(source = 'C:\Users\mihai\JupyterLab\Поебаться\Users.parquet')
UsersData = pd.read_parquet('Users.parquet', engine='fastparquet')
UsersData.columns

UsersData = UsersData[['UserID','UserEmail','TeamID','Name','IsActive','DateRegister']]
UsersData
# UsersData = pd.read_parquet('Users.parquet', engine='fastparquet')
# df.columns
# pd.options.display.max_columns = None

,UserID,UserEmail,TeamID,Name,IsActive,DateRegister
0,52e4b055dd,52e4b055dd@mail.com,6,Janet Garrett,0,2011-08-31
1,844525af93,844525af93@mail.com,10,Dinorah Lindsay,1,2012-05-08
2,e898b47b51,e898b47b51@mail.com,1,Billy Reid,0,2010-01-28
3,e55a5ed02d,e55a5ed02d@mail.com,7,Virginia Fisher,0,2012-09-15
4,f75542e2f8,f75542e2f8@mail.com,10,Debra Cochran,1,2011-05-03
...,...,...,...,...,...,...
4995,2afb4c39f2,2afb4c39f2@mail.com,18,James Moss,0,2017-08-14
4996,42132cd486,42132cd486@mail.com,3,Antonia Tankersley,0,2017-07-31
4997,51ab20581e,51ab20581e@mail.com,4,Bruce Pierson,1,2018-04-22
4998,b1ce0446cd,b1ce0446cd@mail.com,18,Leonard Clark,1,2012-08-11


In [4]:
TeamsData = pd.read_parquet('Teams.parquet', engine='fastparquet')
TeamsData

,TeamID,TeamName,TeamManager,TeamPriority
0,1,None,Camellia Reed,5
1,2,None,Victoria Manning,2
2,3,unlicentious,Mark Todd,5
3,4,complexion,Nancy Schuller,1
4,5,whizziness,Robin Horan,5
5,6,arbute,Norman Mcenany,1
6,7,Simon Zelotes,Clara Lawson,3
7,8,sauterne,Mildred Turner,1
8,9,cercocarpus,Liana Shoemaker,2
9,10,flemingia,Annie Perez,3


In [5]:
MetricsData = pd.read_parquet('Metrics.parquet', engine='fastparquet',)
MetricsData

,UserId,DateMetric,Mails,Hits,SMS,PhoneCalls
0,de20cf2a7a,2018-01-02,21,37,23,14
1,030f56a70e,2019-05-06,16,17,18,9
2,e7c09096d3,2019-01-05,41,44,34,15
3,32da84686c,2019-10-01,32,38,16,4
4,4c56521cef,2016-09-03,54,8,22,1
...,...,...,...,...,...,...
99995,949eb2a242,2019-07-17,35,44,11,6
99996,4e7d46ba18,2016-02-22,49,20,30,13
99997,95733febdf,2019-11-10,39,37,6,8
99998,4f2c60ee36,2019-01-01,3,24,31,5


In [6]:
# asdf = UsersData.set_index('UserID').join(MetricsData.set_index('SMS'))
# asd = MetricsData.join(MetricsData.set_index('UserID'), on='UseriD')
Join = pd.merge(UsersData, MetricsData,  how='inner', left_on=['UserID'], right_on = ['UserId'])
Join
# JoinOne = MetricsData.join(UsersData, lsuffix='1', rsuffix='2')
# joinOne
# JoinTwo = JoinOne.join(TeamsData, lsuffix='3', rsuffix='4')
# JoinTwo
# JoinTwo.keys()

,UserID,UserEmail,TeamID,Name,IsActive,DateRegister,UserId,DateMetric,Mails,Hits,SMS,PhoneCalls
0,52e4b055dd,52e4b055dd@mail.com,6,Janet Garrett,0,2011-08-31,52e4b055dd,2012-02-12,53,0,11,4
1,52e4b055dd,52e4b055dd@mail.com,6,Janet Garrett,0,2011-08-31,52e4b055dd,2013-05-27,49,16,15,15
2,52e4b055dd,52e4b055dd@mail.com,6,Janet Garrett,0,2011-08-31,52e4b055dd,2018-03-07,25,28,29,15
3,52e4b055dd,52e4b055dd@mail.com,6,Janet Garrett,0,2011-08-31,52e4b055dd,2017-10-06,41,47,10,17
4,52e4b055dd,52e4b055dd@mail.com,6,Janet Garrett,0,2011-08-31,52e4b055dd,2012-09-11,21,10,25,17
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,22dd776f25,22dd776f25@mail.com,12,Charles Byrd,0,2015-01-24,22dd776f25,2018-09-20,43,20,24,16
99996,22dd776f25,22dd776f25@mail.com,12,Charles Byrd,0,2015-01-24,22dd776f25,2018-11-08,59,46,22,16
99997,22dd776f25,22dd776f25@mail.com,12,Charles Byrd,0,2015-01-24,22dd776f25,2018-02-08,22,43,18,17
99998,22dd776f25,22dd776f25@mail.com,12,Charles Byrd,0,2015-01-24,22dd776f25,2019-11-15,41,43,31,14


In [7]:
TeamIDRow = pd.merge(TeamsData, Join,  how='inner', left_on=['TeamID'], right_on = ['TeamID'])

TeamIDRow.to_excel('RawData.xlsx')


In [8]:
TeamIDRow.loc[:,'DateMetric'] = pd.to_datetime(
    TeamIDRow['DateMetric'], format='%Y-%m-%d ')
TeamIDRow

,TeamID,TeamName,TeamManager,TeamPriority,UserID,UserEmail,Name,IsActive,DateRegister,UserId,DateMetric,Mails,Hits,SMS,PhoneCalls
0,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2015-09-06,9,7,16,15
1,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2019-01-16,38,21,12,11
2,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2010-08-29,58,16,14,20
3,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2017-09-18,46,28,18,15
4,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2016-07-03,8,38,21,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95235,19,flanking,Renee Facteau,5,53fb37ccc0,53fb37ccc0@mail.com,Henry Riley,1,2010-12-18,53fb37ccc0,2012-12-01,1,21,27,9
95236,19,flanking,Renee Facteau,5,53fb37ccc0,53fb37ccc0@mail.com,Henry Riley,1,2010-12-18,53fb37ccc0,2015-06-22,27,4,23,7
95237,19,flanking,Renee Facteau,5,53fb37ccc0,53fb37ccc0@mail.com,Henry Riley,1,2010-12-18,53fb37ccc0,2016-09-14,4,11,32,8
95238,19,flanking,Renee Facteau,5,53fb37ccc0,53fb37ccc0@mail.com,Henry Riley,1,2010-12-18,53fb37ccc0,2012-10-25,46,29,19,2


In [9]:
TeamIDRow

,TeamID,TeamName,TeamManager,TeamPriority,UserID,UserEmail,Name,IsActive,DateRegister,UserId,DateMetric,Mails,Hits,SMS,PhoneCalls
0,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2015-09-06,9,7,16,15
1,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2019-01-16,38,21,12,11
2,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2010-08-29,58,16,14,20
3,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2017-09-18,46,28,18,15
4,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2016-07-03,8,38,21,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95235,19,flanking,Renee Facteau,5,53fb37ccc0,53fb37ccc0@mail.com,Henry Riley,1,2010-12-18,53fb37ccc0,2012-12-01,1,21,27,9
95236,19,flanking,Renee Facteau,5,53fb37ccc0,53fb37ccc0@mail.com,Henry Riley,1,2010-12-18,53fb37ccc0,2015-06-22,27,4,23,7
95237,19,flanking,Renee Facteau,5,53fb37ccc0,53fb37ccc0@mail.com,Henry Riley,1,2010-12-18,53fb37ccc0,2016-09-14,4,11,32,8
95238,19,flanking,Renee Facteau,5,53fb37ccc0,53fb37ccc0@mail.com,Henry Riley,1,2010-12-18,53fb37ccc0,2012-10-25,46,29,19,2


In [10]:
TeamIDRow = TeamIDRow[(TeamIDRow['DateMetric'] >= '2019-1-1') & (TeamIDRow['DateMetric'] <= '2019-12-31')]
TeamIDRow

,TeamID,TeamName,TeamManager,TeamPriority,UserID,UserEmail,Name,IsActive,DateRegister,UserId,DateMetric,Mails,Hits,SMS,PhoneCalls
1,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2019-01-16,38,21,12,11
7,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2019-04-15,27,41,2,2
8,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2019-06-10,11,37,5,15
17,1,None,Camellia Reed,5,e898b47b51,e898b47b51@mail.com,Billy Reid,0,2010-01-28,e898b47b51,2019-10-19,6,24,11,5
21,1,None,Camellia Reed,5,32232c5aa0,32232c5aa0@mail.com,Glenn Grunder,0,2017-01-22,32232c5aa0,2019-09-06,36,20,33,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95200,19,flanking,Renee Facteau,5,d1ba9f94b6,d1ba9f94b6@mail.com,Joe Hayes,0,2013-04-20,d1ba9f94b6,2019-09-14,60,34,2,5
95208,19,flanking,Renee Facteau,5,d1ba9f94b6,d1ba9f94b6@mail.com,Joe Hayes,0,2013-04-20,d1ba9f94b6,2019-08-11,21,9,27,18
95212,19,flanking,Renee Facteau,5,d1ba9f94b6,d1ba9f94b6@mail.com,Joe Hayes,0,2013-04-20,d1ba9f94b6,2019-05-13,18,25,25,4
95216,19,flanking,Renee Facteau,5,d1ba9f94b6,d1ba9f94b6@mail.com,Joe Hayes,0,2013-04-20,d1ba9f94b6,2019-08-16,48,3,27,18


In [11]:
TeamIDRow.groupby(['TeamManager'], as_index= False).agg({
    'SMS': 'sum'
}).sort_values(by=['SMS'], ascending = False)


,TeamManager,SMS
9,Mark Todd,33637
3,Camellia Reed,29805
17,Theron Radish,28436
18,Victoria Manning,27921
12,Norman Mcenany,27793
7,Liana Shoemaker,27041
2,Bryan Delgado,26988
6,Glenn Montgomery,26934
14,Renee Facteau,26622
4,Clara Lawson,26323
